In [11]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install pillow

In [15]:
pip install keyboard



     -------------------------------------- 58.1/58.1 kB 254.6 kB/s eta 0:00:00


In [24]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2

def open_webcam():
    global photo_image  # Declare photo_image as a global variable

    def update_frame():
        global photo_image  # Declare photo_image as a global variable

        success, frame = cap.read()
        if success:
            pil_image = Image.fromarray(frame)
            photo_image = ImageTk.PhotoImage(pil_image)
            image_item=canvas.create_image(0, 0, anchor=tk.NW, image=photo_image)
            new_frame=ImageTk.PhototImage(photo_image)
            canvas.itemconfigure(image_item, image=photo_image)
        canvas.after(15, update_frame)

    cap = cv2.VideoCapture(0)
    window = tk.Tk()
    canvas = tk.Canvas(window)
    canvas.pack()
    update_frame()
    window.mainloop()

# Call the open_webcam function to open the webcam feed in the tkinter interface
open_webcam()



TclError: image "pyimage1392" doesn't exist

In [14]:
import tkinter as tk
from tkinter import *
from PIL import Image, ImageTk
import cv2

def play_video():
    video_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4")])
    video = cv2.VideoCapture(video_path)
    
    def update_frame():
        ret, frame = video.read()
        if ret:
            # Convert the frame to PIL image
            #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(frame)
            pil_image = pil_image.resize((640, 480))
            
            # Convert the PIL image to Tkinter-compatible format
            tk_image = ImageTk.PhotoImage(pil_image)
            
            # Update the image on the canvas
            canvas.itemconfigure(canvas_image, image=tk_image)
            canvas.image = tk_image  # Store a reference to prevent garbage collection
            
            # Call this function again after a delay to continue playing the video
            canvas.after(30, update_frame)
        else:
            # End of video
            video.release()
    
    # Create the Tkinter window
    window = tk.Tk()
    
    # Create the canvas to display the video frames
    canvas = tk.Canvas(window, width=640, height=480)
    canvas.pack()
    
    # Create a placeholder image on the canvas
    placeholder_image = Image.new("RGB", (640, 480), "black")
    placeholder_tk_image = ImageTk.PhotoImage(placeholder_image)
    canvas_image = canvas.create_image(0, 0, anchor=NW, image=placeholder_tk_image)
    
    # Start playing the video
    update_frame()
    
    # Run the Tkinter event loop
    window.mainloop()

# Call the function to play the video
play_video()


In [1]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np
from ultralytics import YOLO

filepaths=[]
file_list=[]
video_frames=[]
current_file_index=0
classes = ['Gloves', 'Hardhat', 'Mask', 'NO-Gloves', 'NO-Hardhat',
           'NO-Mask', 'NO-Safety Boot', 'NO-Safety Vest', 'Person', 'Safety Boot', 'Safety Vest']
#label = None  # Global variable for label widget
def upload_files():
    global filepaths
    global file_list
    global current_file_index
    global img
    global video_frames
    
    filetypes = [('Image Files', '*.jpg *.jpeg *.png'), ('Video Files', '*.mp4 *.avi')]
    filepaths = filedialog.askopenfilename(multiple=True, filetypes=filetypes)
    
    file_list=[] 
    current_file_index=0
    video_frames=[]
    
    for filepath in filepaths:
        if filepath.lower().endswith(('.jpg', '.jpeg', '.png')):
            img=cv2.imread(filepath)
            
            img=detect(img)
            annotated_image = img[0].plot()
            
            numpy_image = annotated_image.astype(np.uint8)  # Convert to uint8 data type
            numpy_image=numpy_image[:, :, ::-1] #convert from RGB to BGR
            PIL_image = Image.fromarray(numpy_image) #convert numpy array to PIL image

            PIL_image=PIL_image.resize((640,640))
            uploaded_file=ImageTk.PhotoImage(PIL_image)
            file_list.append(uploaded_file)
        else:
            pass
                

    if file_list:
        file_display(current_file_index)
        

def file_display(file_index):
    global label
    global button_forward
    global button_back
    global canvas
    
    #frame3.delete("all")  # Clear the canvas
    #frame3.create_image(0, 0, anchor=NW, image=file_list[file_index])
    
    
    label=Label(image=file_list[file_index])
    label.grid_forget()  # Remove the existing label
    label.grid(in_=frame3, row=0,column=1,columnspan=3) #

    # Update the state of forward and back buttons
    if file_index == 0:
        button_back.config(state=tk.DISABLED)
    else:
        button_back.config(state=tk.NORMAL)
        
    if file_index == len(file_list) - 1:
        button_forward.config(state=tk.DISABLED)
    else:
        button_forward.config(state=tk.NORMAL)    
    
def forward():
    global current_file_index
    current_file_index +=1
    file_display(current_file_index)
    
def back():
    global current_file_index
    current_file_index -=1
    file_display(current_file_index)


def open_webcam():
    
    # Load the YOLOv8 model
    #model = YOLO('best.pt')

    # Open the video file
    cap = cv2.VideoCapture(0)

    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            
            # Run YOLOv8 inference on the frame
            result_frame = detect(frame)

            # Visualize the results on the frame
            annotated_frame = result_frame[0].plot()
            

            # Display the annotated frame
            cv2.imshow("YOLOv8 Inference", annotated_frame)
            
            # Process the frame for each class
            #for class_index in range(len(classes)):
                #process_image(class_index, frame)
            #pil_image = Image.fromarray(annotated_frame)
            #pil_image = pil_image.resize((640, 640))  # Resize the image to fit the canvas

            # Convert PIL Image to PhotoImage
            #photo_image = ImageTk.PhotoImage(pil_image)

            # Update the canvas with the new image
            #canvas.create_image(0, 0, anchor=NW, image=photo_image)

            
            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break

    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()

def detect(image, class_index=None):
    model = YOLO('best.pt')
    if class_index is not None and class_index < len(classes):
        selected_class = classes[class_index]
        result = model.predict(image, classes=classes.index(selected_class), verbose=False)
    else:
        result = model(image, verbose=False)
    return result

def process_image(class_index):

    selected_class = classes[class_index]

    for i, image_path in enumerate(filepaths):
        if image_path.lower().endswith(('.jpg', '.jpeg', '.png')):
            img = cv2.imread(image_path)
            result = detect(img, class_index)
            annotated_image = result[0].plot()
            numpy_image = annotated_image.astype(np.uint8)
            numpy_image = numpy_image[:, :, ::-1]
            pil_image = Image.fromarray(numpy_image)
            pil_image = pil_image.resize((640, 640))
            uploaded_file = ImageTk.PhotoImage(pil_image)
            file_list[i] = uploaded_file
    file_display(current_file_index)



def all_class_process(class_index):
    #selected_class = classes[class_index]
    for i, image_path in enumerate(filepaths):
        if image_path.lower().endswith(('.jpg', '.jpeg', '.png')):
            img = cv2.imread(image_path)
            result = detect(img, class_index)
            annotated_image = result[0].plot()
            numpy_image = annotated_image.astype(np.uint8)
            numpy_image = numpy_image[:, :, ::-1]
            pil_image = Image.fromarray(numpy_image)
            pil_image = pil_image.resize((640, 640))
            uploaded_file = ImageTk.PhotoImage(pil_image)
            file_list[i] = uploaded_file
    file_display(current_file_index)
# Create the tkinter window
window = tk.Tk()
window.title('Construction Site Safety Instruction')
window.iconbitmap('safety-icon.ico')

#label=Label()

frame2 = LabelFrame(window,text='control Frame ',padx=10,pady=10)
frame2.grid(sticky= "N")

frame1=LabelFrame(window,text='Control Detection', padx=10,pady=10)
frame1.grid(row=1, column=0,sticky= "N")

frame3 = LabelFrame(window,text='Main Frame', width=640, height=640)
frame3.grid(row=0, column=1, columnspan=3, rowspan=8)

# Create the buttons
upload_button = tk.Button(frame2, text="Upload Files", command=upload_files, height=1, width=15)
upload_button.grid()

webcam_button = tk.Button(frame2, text="Open Webcam", command=open_webcam, height=1, width=15)
webcam_button.grid()


button_back = tk.Button(window, text="<<", command=back, state=tk.DISABLED)
button_exit = tk.Button(window, text="Exit Program", command=window.destroy)
button_forward = tk.Button(window, text=">>", command=forward)

button_back.grid(row=9, column=1)
button_exit.grid(row=9, column=2)
button_forward.grid(row=9, column=3)

# Create the buttons for each class
for i in range(len(classes)):
    class_name = classes[i]
    button = tk.Button(frame1, text=class_name, command=lambda class_index=i: process_image(class_index), height=1, width=15)
    button.grid(row=i+1, rowspan=1)
button = tk.Button(frame1, text="All Classes", command=lambda class_index=None: all_class_process(class_index), height=1, width=15)
button.grid(row=0, column=0)

# Run the tkinter event loop
window.mainloop()
    


In [1]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('best.pt')

# Open the video file
cap = cv2.VideoCapture(0)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame,verbose=False)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


In [ ]:
Thank you for reaching out. To customize the colors for the plot generated by res[0].plot(), you can pass a dictionary of colors to the plot() function. The keys should correspond to the label names and the values should be the desired color codes.

For instance, assuming you have two classes "cat" and "dog" and you want to plot them in red and blue respectively, you can do the following:

colors = {"cat": "red", "dog": "blue"}
res_plotted = res[0].plot(colors=colors)
Does that help? Let me know if you have any other questions.

Best regards.

In [9]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2

video_files = []
current_video_index = 0
playing = False
video = None

def play_video():
    global video_files
    global current_video_index
    global video
    
    video_files = filedialog.askopenfilenames(filetypes=[("Video Files", "*.mp4")])
    
    if len(video_files) == 0:
        return  # No video files selected
    
    current_video_index = 0
    playing = False
    
    def update_frame():
        global current_video_index
        nonlocal playing
        
        if not playing:
            return
        
        video_path = video_files[current_video_index]
        video = cv2.VideoCapture(video_path)
        
        def get_frame():
            ret, frame = video.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                pil_image = Image.fromarray(frame)
                pil_image = pil_image.resize((640, 480))
                return ImageTk.PhotoImage(pil_image)
            else:
                video.release()
                return None
        
        def display_frame():
            global playing
            if playing:
                frame = get_frame()
                if frame is not None:
                    canvas.itemconfigure(canvas_image, image=frame)
                    canvas.image = frame
                    window.after(30, display_frame)
                else:
                    current_video_index += 1
                    if current_video_index < len(video_files):
                        update_frame()
                    else:
                        current_video_index = 0  # Reset index to play videos again
                        playing = False
        
        display_frame()
    
    def previous_video():
        global current_video_index
        if current_video_index > 0:
            current_video_index -= 1
            if playing:
                update_frame()
    
    def next_video():
        global current_video_index
        if current_video_index < len(video_files) - 1:
            current_video_index += 1
            if playing:
                update_frame()
    
    def toggle_playback():
        global playing
        playing = not playing
        if playing:
            play_button.config(text="Pause")
            if current_video_index < len(video_files):
                update_frame()
        else:
            play_button.config(text="Play")
    
    window = tk.Tk()
    
    canvas = tk.Canvas(window, width=640, height=480)
    canvas.pack()
    
    placeholder_image = Image.new("RGB", (640, 480), "black")
    placeholder_tk_image = ImageTk.PhotoImage(placeholder_image)
    canvas_image = canvas.create_image(0, 0, anchor=NW, image=placeholder_tk_image)
    
    previous_button = tk.Button(window, text="<< Previous", command=previous_video)
    previous_button.pack(side=LEFT)
    
    play_button = tk.Button(window, text="Play", command=toggle_playback)
    play_button.pack(side=LEFT)
    
    next_button = tk.Button(window, text="Next >>", command=next_video)
    next_button.pack(side=LEFT)
    
    window.mainloop()

# Call the function to play the video
play_video()


In [ ]:
The update_frame function is responsible for updating the video frame on the canvas. It reads a frame from the video using video.read(). If a frame is successfully read (ret is True), it converts the frame from BGR to RGB color space, resizes it to the desired dimensions (640x480), and creates a PIL image from the frame.

The PIL image is then converted to a Tkinter-compatible format (ImageTk.PhotoImage) and assigned to the tk_image variable. The canvas.itemconfigure function is used to update the image on the canvas by specifying the canvas_image created earlier and setting its image attribute to tk_image. A reference to the Tkinter image is also stored in canvas.image to prevent garbage collection.

Finally, the update_frame function is scheduled to be called again after a delay of 30 milliseconds using canvas.after(30, update_frame). This creates a continuous loop, updating the frames on the canvas until the end of the video is reached. When the end of the video is reached (ret is False), the video.release() is called to release the video capture object.

In the main part of the code, a Tkinter window (window) is created using tk.Tk(). Then, a canvas (canvas) is created with a width and height of 640x480 pixels and packed into the window using canvas.pack().

A placeholder image (placeholder_image) is created with a black background and the same dimensions as the canvas. It is then converted to a Tkinter image (placeholder_tk_image) using ImageTk.PhotoImage.

On the canvas, an image item (canvas_image) is created using canvas.create_image. The image item is anchored to the northwest (NW) corner of the canvas, and the placeholder Tkinter image is assigned as its initial image.

The update_frame function is called to start playing the video by updating the frame on the canvas. Finally, the Tkinter event loop is started using window.mainloop(), which allows the GUI to be displayed and responds to user interactions.

I hope this explanation helps you understand the code better! Let me know if you have any further questions.

In [5]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np
from ultralytics import YOLO

filepaths = []
file_list = []
video_frames = []
current_file_index = 0
classes = ['Gloves', 'Hardhat', 'Mask', 'NO-Gloves', 'NO-Hardhat',
           'NO-Mask', 'NO-Safety Boot', 'NO-Safety Vest', 'Person', 'Safety Boot', 'Safety Vest']

def upload_files():
    global filepaths
    global file_list
    global current_file_index
    global video_frames

    filetypes = [('Image Files', '*.jpg *.jpeg *.png'), ('Video Files', '*.mp4 *.avi')]
    filepaths = filedialog.askopenfilename(multiple=True, filetypes=filetypes)

    file_list = []
    current_file_index = 0
    video_frames = []

    for filepath in filepaths:
        if filepath.lower().endswith(('.jpg', '.jpeg', '.png')):
            img = cv2.imread(filepath)
            img = detect(img)
            annotated_image = img[0].plot()
            numpy_image = annotated_image.astype(np.uint8)
            numpy_image = numpy_image[:, :, ::-1]
            pil_image = Image.fromarray(numpy_image)
            pil_image = pil_image.resize((640, 640))
            uploaded_file = ImageTk.PhotoImage(pil_image)
            file_list.append(uploaded_file)
        else:
            pass

    if file_list:
        file_display(current_file_index)

def file_display(file_index):
    global label
    global button_forward
    global button_back
    global canvas

    canvas.delete("all")  # Clear the canvas
    canvas.create_image(0, 0, anchor=NW, image=file_list[file_index])

    # Update the state of forward and back buttons
    if file_index == 0:
        button_back.config(state=tk.DISABLED)
    else:
        button_back.config(state=tk.NORMAL)

    if file_index == len(file_list) - 1:
        button_forward.config(state=tk.DISABLED)
    else:
        button_forward.config(state=tk.NORMAL)

def forward():
    global current_file_index
    current_file_index += 1
    file_display(current_file_index)

def back():
    global current_file_index
    current_file_index -= 1
    file_display(current_file_index)

def detect(image, class_index=None):
    model = YOLO('best.pt')
    if class_index is not None and class_index < len(classes):
        selected_class = classes[class_index]
        result = model.predict(image, classes=classes.index(selected_class), verbose=False)
    else:
        result = model(image, verbose=False)
    return result

def process_image(class_index):
    selected_class = classes[class_index]
    for i, image_path in enumerate(filepaths):
        if image_path.lower().endswith(('.jpg', '.jpeg', '.png')):
            img = cv2.imread(image_path)
            result = detect(img, class_index)
            annotated_image = result[0].plot()
            numpy_image = annotated_image.astype(np.uint8)
            numpy_image = numpy_image[:, :, ::-1]
            pil_image = Image.fromarray(numpy_image)
            pil_image = pil_image.resize((640, 640))
            uploaded_file = ImageTk.PhotoImage(pil_image)
            file_list[i] = uploaded_file
    file_display(current_file_index)
def all_class_process(class_index):
    #selected_class = classes[class_index]
    for i, image_path in enumerate(filepaths):
        if image_path.lower().endswith(('.jpg', '.jpeg', '.png')):
            img = cv2.imread(image_path)
            result = detect(img, class_index)
            annotated_image = result[0].plot()
            numpy_image = annotated_image.astype(np.uint8)
            numpy_image = numpy_image[:, :, ::-1]
            pil_image = Image.fromarray(numpy_image)
            pil_image = pil_image.resize((640, 640))
            uploaded_file = ImageTk.PhotoImage(pil_image)
            file_list[i] = uploaded_file
    file_display(current_file_index)
# Create the tkinter window
window = tk.Tk()
window.title('Construction Site Safety Instruction')
window.iconbitmap('safety-icon.ico')

# Create the buttons
upload_button = tk.Button(window, text="Upload Files", command=upload_files)
upload_button.grid()

# Create the canvas
canvas = tk.Canvas(window, width=640, height=640)
canvas.grid(row=3, column=0, columnspan=3)

button_back = tk.Button(window, text="<<", command=back, state=tk.DISABLED)
button_exit = tk.Button(window, text="Exit Program", command=window.quit)
button_forward = tk.Button(window, text=">>", command=forward)

button_back.grid(row=4, column=0)
button_exit.grid(row=4, column=1)
button_forward.grid(row=4, column=2)

# Create the buttons for each class
for i in range(len(classes)):
    class_name = classes[i]
    button = tk.Button(window, text=class_name, command=lambda class_index=i: process_image(class_index))
    button.grid(row=5, column=i)
button = tk.Button(window, text="All Classes", command=lambda class_index=None: all_class_process(class_index))
button.grid(row=6, column=1)

# Run the tkinter event loop
window.mainloop()
